In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
from torch.utils.data import DataLoader, IterableDataset
import os
import torch.nn as nn
import numpy as np
import pandas as pd
from dask import delayed
from torch.autograd import Variable
from os.path import join
from tqdm import tqdm
import random
import tensorflow as tf 
import os
from collections import Counter
from functools import reduce
import operator
import json 

Using TensorFlow backend.


In [0]:
train_path = '/content/drive/My Drive/TransT/train.txt'

In [0]:
types_path = '/content/drive/My Drive/TransT/newType.txt'

In [0]:
entities_to_index_path = "/content/drive/My Drive/TransT/entities_to_index.json"

In [0]:
relations_to_index_path = "/content/drive/My Drive/TransT/relations_to_index.json"

In [0]:
class TrainDataset:
    """
    Класс нашего датасета. С помошью него для модели TransT реализован Negative
    Sampling. 
    """
    def __init__(self, train_path, types_path, entities_to_index_path, relations_to_index_path):
        """
        train_path - путь до файла, к котором хранятся данные для обучения 
        train_path - путь до файла, к котором хранятся данные о типах для каждого entity
        entities_to_index_path - путь до файла, где хранится то, как переводить ссылки на entity в целочисленные индексы
        relations_to_index_path - путь до файла, где хранится то, как переводить ссылки на relation в целочисленные индексы
        """
        self.train_path = train_path
        self.types_path = types_path
        self.entities_to_index_path = entities_to_index_path
        self.relations_to_index_path = relations_to_index_path
        
        self.entities_to_index, self.relations_to_index = self.create_converters()

        self.head_rel_to_tail = {}
        self.tail_rel_to_head = {}
        self.head_tail_to_rel = {}

        self.pos_triplets = self.generate_positive_triplets()
        self.neg_triplets = self.generate_negative_triplets()

        self.types_dict = self.generate_types_dict()

    def create_converters(self):
        """
        Считываем файлы и передаем их в словари.
        Выводит: кортеж из 2 словарей(entities_to_index, relations_to_index)
        entities_to_index: словарь, ключами которого являются ссылки на entity, а значениями - целочисленный индекс
        relations_to_index: словарь, ключами которого являются ссылки на relation, а значениями - целочисленный индекс
        """
        with open(self.entities_to_index_path, 'r') as f:
            for el in f:
                entities_to_index = json.loads(json.loads(el))

        with open(self.relations_to_index_path, 'r') as f:
            for el in f:
                relations_to_index = json.loads(json.loads(el))
        
        return entities_to_index, relations_to_index

    def generate_positive_triplets(self):
        """
        Считываем файл и группируем его содержимое по триплетам. 
        Выводит: np.ndarray размером (n, 3), в котором каждая строка отражает связь. Каждая строка состоит из head, rel, tail. 

        Кроме того, мы заполняем self.head_rel_to_tail, tail_rel_to_head и head_tail_to_rel для того, 
        чтобы понимать какие ссылки были для сочетаний head и rel, tail и rel ну и head и tail соответственно. 
        Это нужно для того, чтобы нормально проводить Negative Sampling. Если выбирать значения для Negative Sampling просто 
        на рандом, то есть вероятность, что попадется рандомное число, которое на самом деле не Negative. Для избежания 
        сего казуса мы и заполняем эти 3 словаря словарей. 
        """
        t = []

        with open(self.train_path, 'r') as f:
            data = f.readlines()

            for el in tqdm(data):
                head, rel, tail = [str(a) for a in el.split()]
                head = self.entities_to_index[head]
                rel = self.relations_to_index[rel]
                tail = self.entities_to_index[tail]

                t.append([head, rel, tail])


                # Fill the heads, rels and tails into the dictionary of dictionaries head_rel_to_tail
                if head not in self.head_rel_to_tail.keys():
                    self.head_rel_to_tail[head] = {rel: [tail]}
                else:
                    if rel not in self.head_rel_to_tail[head].keys():
                        self.head_rel_to_tail[head][rel] = [tail]
                    else:
                        self.head_rel_to_tail[head][rel].append(tail)
                        
                # Fill the tails, rels and heads into the dictionary of dictionaries tail_rel_to_head
                if tail not in self.tail_rel_to_head.keys():
                    self.tail_rel_to_head[tail] = {rel: [head]}
                else:
                    if rel not in self.tail_rel_to_head[tail]:
                        self.tail_rel_to_head[tail][rel] = [head]
                    else:
                        self.tail_rel_to_head[tail][rel].append(head)

                # Fill the heads, tails and rels into the dictionary of dictionaries head_tail_to_rel
                if head not in self.head_tail_to_rel.keys():
                    self.head_tail_to_rel[head] = {tail: [rel]}
                else:
                    if tail not in self.head_tail_to_rel[head].keys():
                        self.head_tail_to_rel[head][tail] = [rel]
                    else:
                        self.head_tail_to_rel[head][tail].append(rel)

        pos_triplets = np.array(t)

        self.num_ent = len(self.entities_to_index)
        self.num_rel = len(self.relations_to_index)               

        
        return pos_triplets      


    def generate_negative_triplets(self):
        """
        Создаем Negative Samples. 
        Выводит: np.ndarray размером (n, 3, 3). Для каждой строки из self.pos_triplets формируется целых 3 Negative Samples, 
        поэтому размер не (n, 3).  
        """
        n = []

        for i in tqdm(range(self.pos_triplets.shape[0])):
            head, rel, tail = self.pos_triplets[i]

            neg_head = np.random.randint(0, self.num_ent)
            neg_tail = np.random.randint(0, self.num_ent)
            neg_rel = np.random.randint(0, self.num_rel)

            while neg_tail in self.head_rel_to_tail[head][rel] or neg_tail == tail:
                neg_tail = np.random.randint(0, self.num_ent)
            
            while neg_head in self.tail_rel_to_head[tail][rel] or neg_head == head:
                neg_head = np.random.randint(0, self.num_ent)

            while neg_rel in self.head_tail_to_rel[head][tail] or neg_rel == rel:
                neg_rel = np.random.randint(0, self.num_rel)

            n.append([[neg_head, rel, tail], [head, neg_rel, tail], [head, rel, neg_tail]])

        return np.array(n)

    def generate_types_dict(self):
        """
        Создаем словарь, ключами котрого являются целочисленные представления ссылок на entity, а значениями - списки типов для этих entity
        """
        d = {}
        with open(self.types_path, 'r') as f:
            data = f.readlines()
            for el in tqdm(data):
                d[self.entities_to_index[el.split()[0]]] = el.split()[1:]

        return d

    def generate_types_sets(self, p=0.05):
        """
        Для расчета prior probability нам нужно знать, какме типы наиболее часто встречаются в head или tail для определенного rel. 
        Именно для этого мы и создаем types_in_head и types_in_tail. 
        types_in_head, types_in_tail: словари словарей, в которых хранятся частоты вхождений определенных типов в head или tail  для определенных отношений. 
        """
        types_in_head =  {}
        types_in_tail = {}

        for rel in tqdm(self.relations_to_index.values()):
            t1 = []
            t2 = []
            data = self.pos_triplets[self.pos_triplets[:, 1] == rel]
            for el in data:
                for type_ in self.types_dict[el[0]]:
                    t1.append(type_)

                for type_ in self.types_dict[el[2]]:
                    t2.append(type_)

            c_head = Counter(t1)
            c_tail = Counter(t2)

            num_head = len(t1)
            num_tail = len(t2)

            types_in_head[rel] = {k: v / num_head for k, v in c_head.items() if v / num_head > p}
            types_in_tail[rel] = {k: v / num_tail for k, v in c_tail.items() if v / num_tail > p}


        return types_in_head, types_in_tail

                

    def __getitem__(self, index):
        return self.pos_triplets[index, :], self.neg_triplets[index, :, :]

    def __len__(self):
        return self.pos_triplets.shape[0]

In [10]:
a = TrainDataset(train_path, types_path, entities_to_index_path, relations_to_index_path)

100%|██████████| 14951/14951 [00:00<00:00, 253049.84it/s]


In [11]:
h, t = a.generate_types_sets(p=0.05)

100%|██████████| 1345/1345 [00:06<00:00, 204.38it/s]


In [0]:
class TransT(nn.Module):
    """
    Наша модель, которая будет обучать ембеддинги. 
    Требует доработки
    """
    def __init__(self, train_dataset, vector_length=200): 
        super(TransT, self).__init__()
        self.train_dataset = train_dataset
        self.vector_length = vector_length
        self.entity_emb = self.generate_embedddings()
        self.relation_emb = nn.Embedding(self.train_dataset.num_rel, self.vector_length)

    def generate_embedddings(self):
        ls = []
        for i in tqdm(range(self.train_dataset.num_ent)):
            ls.append(nn.Embedding(len(self.train_dataset.types_dict[i]), self.vector_length))

        return ls 

    # def forward(self, pos_triplets, negative_triplets):
    #     # This method will be written soon 
    #     print(pos_triplets.shape, neg_triplets.shape)

In [20]:
model = TransT(a)

100%|██████████| 14951/14951 [00:01<00:00, 10474.94it/s]
